# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Import dependencies

import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import json
import requests
import scipy.stats as st
import requests as req
import pandas as pd
import requests
import gmaps
import os
from pprint import pprint
from datetime import datetime
from citipy import citipy

# API Keys needed
from api_keys import weather_api_key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Path to read the City_Weather_Dataframe.csv file
file = os.path.join("..", "WeatherPy", "City_Weather_Dataframe.csv")

#Read file and store into a dataframe
city_weather_df=pd.read_csv(file)


In [4]:
city_weather_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed,Country,Date
0,Eureka,40.80,-124.16,63.00,100,97,1.12,US,1600350659
1,Verāval,20.90,70.37,82.17,78,94,10.83,IN,1600350870
2,Rikitea,-23.12,-134.97,71.94,82,94,21.63,PF,1600350871
3,Vila Velha,-20.33,-40.29,81.00,57,20,16.11,BR,1600350642
4,Ushuaia,-54.80,-68.30,37.40,59,75,39.15,AR,1600350873


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Create location based on lat / long
lat_long = city_weather_df[['Latitude', 'Longitude']]

# Create humidity
humidity = city_weather_df['Humidity (%)']

In [6]:
figure_layout = {'width': '900px', 
                 'height': '600px', 
                 'border': '1px solid black', 
                 'padding': '1px',
                 'margin': '0 auto 0 auto'}

# Create heatmap
heatmap_fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
heatmap_layer = gmaps.heatmap_layer(lat_long, weights=humidity, 
                                    dissipating=False, max_intensity=100, point_radius=2)

# Add heat layers
heatmap_fig.add_layer(heatmap_layer)

heatmap_fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the DataFrame for be based on ideal weather conditions. Guidelines provided include:
# A max temperature between 70 - 80 degrees. 
# Wind speed less than 15 mph. Increased wind temperature criteria to 15 mph to increase number of cities from 12 to 17
# Zero cloudiness
# Any city that does not meet these 3 criteria will be dropped. 

# Create final DataFrame
# ideal_weather_df = pd.DataFrame(city_weather_df, columns = ["City", "Max Temperature (F)", "Cloudiness", "Wind Speed"])
ideal_weather_df = pd.DataFrame(city_weather_df)

# Select cities that only have ideal weather conditions as listed above
temperature = (ideal_weather_df["Max Temperature (F)"] <= 80) &  (ideal_weather_df["Max Temperature (F)"] > 70) 
cloudiness = ideal_weather_df["Cloudiness"] == 0
wind = ideal_weather_df["Wind Speed"] < 15

ideal_weather_df = ideal_weather_df[temperature & cloudiness & wind]
ideal_weather_df


,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed,Country,Date
6,Cape Town,-33.93,18.42,71.01,55,0,6.93,ZA,1600350876
69,Faanui,-16.48,-151.75,77.90,77,0,13.35,PF,1600350961
96,Bereket,39.24,55.52,73.56,22,0,4.43,TM,1600350998
180,Santiago del Estero,-27.80,-64.26,76.57,31,0,7.27,AR,1600351118
194,Rafaela,-31.25,-61.49,71.01,25,0,3.00,AR,1600351137
214,Graaff-Reinet,-32.25,24.53,73.99,29,0,7.00,ZA,1600351162
230,Mossamedes,-15.20,12.15,70.03,61,0,11.61,AO,1600351183
236,San Quintín,30.48,-115.95,77.49,44,0,7.54,MX,1600351190
263,Gumdag,39.21,54.59,78.39,17,0,5.91,TM,1600351238
270,Roebourne,-20.78,117.13,75.79,50,0,9.95,AU,1600351247


In [8]:
ideal_weather_df.count()

City                   18
Latitude               18
Longitude              18
Max Temperature (F)    18
Humidity (%)           18
Cloudiness             18
Wind Speed             18
Country                18
Date                   18
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather_df

In [10]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name']= ""
hotel_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed,Country,Date,Hotel Name
6,Cape Town,-33.93,18.42,71.01,55,0,6.93,ZA,1600350876,
69,Faanui,-16.48,-151.75,77.90,77,0,13.35,PF,1600350961,
96,Bereket,39.24,55.52,73.56,22,0,4.43,TM,1600350998,
180,Santiago del Estero,-27.80,-64.26,76.57,31,0,7.27,AR,1600351118,
194,Rafaela,-31.25,-61.49,71.01,25,0,3.00,AR,1600351137,


In [11]:
# Create a list to store Hotel Names
hotel_name = []

# Dictionary to store query parameters
params = {
    "radius": 5000,
    "type": "Hotel",
    "keyword": "Hotel",
    "key": g_key,}

# Use the latitude / longitude coordinates to identify hotels
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"]=f"{lat},{lng}"
    
# Search for Hotels at the above lat / long coordinates
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
# Make request and print url
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try: 
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)


In [12]:
hotel_name

['Atlanticview Cape Town Boutique Hotel',
 'InterContinental Bora Bora Resort Thalasso Spa',
 'Railway motel',
 'Hotel Savoy',
 'Amérian Rafaela Hotel',
 'Drostdy Hotel',
 'iu Hotel Namibe',
 'Old Mill Hotel',
 nan,
 'Latitude20 Roebourne Village',
 nan,
 'Plaza Inn Kendari by Horison',
 '哈密环球大酒店',
 nan,
 'Le Relais de Singani hôtel',
 'Hotel Fanilo',
 'Hollins',
 'Tianyuan International Hotel']

In [13]:
hotel_df['Hotel Name']= hotel_name
hotel_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed,Country,Date,Hotel Name
6,Cape Town,-33.93,18.42,71.01,55,0,6.93,ZA,1600350876,Atlanticview Cape Town Boutique Hotel
69,Faanui,-16.48,-151.75,77.90,77,0,13.35,PF,1600350961,InterContinental Bora Bora Resort Thalasso Spa
96,Bereket,39.24,55.52,73.56,22,0,4.43,TM,1600350998,Railway motel
180,Santiago del Estero,-27.80,-64.26,76.57,31,0,7.27,AR,1600351118,Hotel Savoy
194,Rafaela,-31.25,-61.49,71.01,25,0,3.00,AR,1600351137,Amérian Rafaela Hotel


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol overlay
hotel_layer = gmaps.marker_layer(locations, hotel_info)

marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add layers
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…